In [36]:
import json
import pickle as pkl
from sim.healthcare import *
import numpy as np
import numpy.random as rd

In [6]:
pars_dx = json.load(open('pars/pars_cdx.json', 'r'))
pars_tx = json.load(open('pars/pars_tx.json', 'r'))
prev = pars_tx['prev']
pars_tx = pars_tx['pars']

In [10]:
len(pars_tx), len(pars_dx)

(2000, 4000)

In [37]:
prev

{'PrevUt': 0.0020096,
 'PrevAsym': 0.0012318,
 'PrevSym': 0.00042218,
 'PrevExCS': 0.00035562,
 'TBLikeUt': 0.04285987,
 'PrCSIPub': 0.48281849}

In [11]:
pars_tx[0]

{'ppm': 0.54819008,
 'ppv_pri': 0.70076372,
 'txi_pub': 0.00091004,
 'txi_eng': 0.00030101,
 'dur_pri': 0.91158053,
 'p_pri_on_pub': 0.28116338,
 'txi_pri': 0.00024809,
 'ppv_eng': 0.70076372,
 'tx_pri': 0.0006628,
 'drug_time': 0.0006042,
 'p_pub': 0.62368179,
 'p_under': 0.1700243,
 'tp_pri_drug': 642411.96131784,
 'tp_pri_drug_time': 585610.23387954,
 'tp_pri_txi': 759469.778308,
 'lp__': -548.84132338,
 'ppv_pub': 0.75}

In [64]:
particle = dict()

particle.update({k: v for k, v in p_dx.items() if k.startswith('sens_') or k.startswith('spec_')})
particle['p_scanty'] = p_dx['p_scanty']
particle['dur_pub'] = 0.5
particle['dur_pri'] = p_tx['dur_pri']
particle['p_pri_on_pub'] = p_tx['p_pri_on_pub']
particle['system'] = sys = get_system(pars_dx[0], pars_tx[0])

particle['p_dx'] = np.array([sys.Public.seek_care(1, 0).TruePos, sys.Engaged.seek_care(1, 0).TruePos, sys.Private.seek_care(1, 0).TruePos])
particle['p_txi'] = np.array([0.95, 0.85, 0.85])
particle['p_ent'] = sys.Entry
particle['p_itt'] = 1 / (2.7 * particle['p_dx'] * particle['p_txi'] * particle['p_ent']).sum()
particle['txi'] = np.array([p_tx['txi_pub'], p_tx['txi_eng'], p_tx['txi_pri']]).sum()

particle

{'sens_s': 0.6313,
 'sens_x': 0.8367,
 'sens_x_sn': 0.6355,
 'spec_s': 0.9772,
 'spec_x': 0.9719,
 'spec_x_sn': 0.9743,
 'sens_cdx': 0.7821,
 'spec_cdx': 0.9643,
 'sens_cdx_bn': 0.7821,
 'spec_cdx_bn': 0.9643,
 'p_scanty': 0.1695,
 'dur_pub': 0.5,
 'dur_pri': 0.91158053,
 'p_pri_on_pub': 0.28116338,
 'system': <sim.healthcare.system.System at 0x7f6569a34250>,
 'p_dx': array([0.93591796, 0.79992145, 0.7821    ]),
 'p_txi': array([0.95, 0.85, 0.85]),
 'p_ent': array([0.55644093, 0.24067886, 0.20288021]),
 'p_itt': 0.4668958704869209,
 'txi': 0.00145914}

In [73]:
with open('d.pkl', 'wb') as f:
    pkl.dumps(particle)

b'\x80\x04\x95\xbe\x05\x00\x00\x00\x00\x00\x00}\x94(\x8c\x06sens_s\x94G?\xe43\x9c\x0e\xbe\xdf\xa4\x8c\x06sens_x\x94G?\xea\xc6?\x14\x12\x05\xbc\x8c\tsens_x_sn\x94G?\xe4V\x04\x18\x93t\xbc\x8c\x06spec_s\x94G?\xefE8\xef4\xd6\xa1\x8c\x06spec_x\x94G?\xef\x19\xce\x07_o\xd2\x8c\tspec_x_sn\x94G?\xef-w1\x8f\xc5\x05\x8c\x08sens_cdx\x94G?\xe9\x06\xf6\x94Fs\x82\x8c\x08spec_cdx\x94G?\xee\xdb\x8b\xacq\x0c\xb3\x8c\x0bsens_cdx_bn\x94G?\xe9\x06\xf6\x94Fs\x82\x8c\x0bspec_cdx_bn\x94G?\xee\xdb\x8b\xacq\x0c\xb3\x8c\x08p_scanty\x94G?\xc5\xb2-\x0eV\x04\x19\x8c\x07dur_pub\x94G?\xe0\x00\x00\x00\x00\x00\x00\x8c\x07dur_pri\x94G?\xed+\xaa\xee\x80\xa6\xaf\x8c\x0cp_pri_on_pub\x94G?\xd1\xfe\x94\xb0{\xb3U\x8c\x06system\x94\x8c\x15sim.healthcare.system\x94\x8c\x06System\x94\x93\x94)\x81\x94}\x94(\x8c\x05Entry\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x03\x85\x94h\x19\x8c\x05dtype\x94\x93\x94\x8c\x02f8\x94\x